In [3]:
# Library Imports

from pyspark.mllib.regression import LabeledPoint, RidgeRegressionWithSGD

from numpy import array

from pyspark import SparkContext

from pyspark.mllib.feature import StandardScaler

# Load and parse the data

def parsePoint(line):

    values = [float(x) for x in line.split(',')]

    return LabeledPoint(values[11], values[0:10])

sc = SparkContext("local", "RidgeWithSGD")

sparkHome = os.environ.get('SPARK_HOME')
fileLocation = sparkHome + "/Assignment1/data.csv";

data = sc.textFile(fileLocation)

parsedData = data.map(parsePoint)

#Split training/test sets

(trainingData, testData) = parsedData.randomSplit([0.6, 0.4])

#Scaling features 

label = trainingData.map(lambda x: x.label)

features = trainingData.map(lambda x: x.features)

scaler1 = StandardScaler().fit(features)

trainingData = label.zip(scaler1.transform(features))

trainingData = trainingData.map(lambda (x,y) : LabeledPoint(x,y))

# Build the model with the parsedData for 100 iterations using regularization 

#parameter as 0.01 and regularization type as ‘l2’

model = RidgeRegressionWithSGD.train(trainingData,iterations=100,step=0.0001,regParam=0.1)

# Evaluate the model on training data

valuesAndPreds = testData.map(lambda p: (p.label, model.predict(p.features)))

#Calculating the Sqrt Mean Square Error using the following formula

MSE = valuesAndPreds.map(lambda (v, p): (v - p)**2).reduce(lambda x, y: x + y) / valuesAndPreds.count()

MSER = (MSE)**0.5

#Calculating the Variance using the following formula

meanVal = valuesAndPreds.map(lambda (v,p): p).sum() /valuesAndPreds.count()

variance = valuesAndPreds.map(lambda (v,p): (p - meanVal)**2).sum() /valuesAndPreds.count()

#Print the Sqrt Mean Square Error and Variance calculated

print("Mean Squared Root Error =" +str(MSER))

print ("Variance = " + str(variance))

Mean Squared Root Error =0.183485114139
Variance = 7.99995817429e-07


In [2]:
sc.stop()